# Proyecto Open Data I
## Radares, y su eficiencia en la CAM
### Recopilación, limpieza y tratamiento de los datos
Este cuaderno pretende enseñar el proceso de limpieza de los datos relativos a los radares en la CAM
_Paula Gómez Lucas, Alejandro Majado Martínez_

In [ ]:
# Importar librerías
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import numpy as np
from pandas_summary import DataFrameSummary
import textwrap

from scipy.stats import chisquare

A continuación, se muestra la clase que está compuesta de todos los métodos que se encargan de la limpieza y transformación de los datos

In [ ]:
class CSVDataLoader:
    """
    A class for loading and cleaning CSV data from a specified folder path.

    Attributes:
    -----------
    folder_path : str
        The path to the folder containing the CSV files to be loaded.

    data : dict
        A dictionary containing the loaded CSV data, where the keys are the file names and the values are the corresponding dataframes.
    """

    def __init__(self, folder_path):
        """
        Initializes a CSVDataLoader object with the specified folder path.

        Parameters:
        -----------
        folder_path : str
            The path to the folder containing the CSV files to be loaded.
        """
        self.folder_path = folder_path
        self.data = {}
        self.filename = []
        self.keys = []

    def load_data(self):
        """
        Loads CSV data from the specified folder path into a dictionary.

        Returns:
        --------
        None
        """
        csv_files = [f for f in os.listdir(self.folder_path) if f.endswith('.csv')]
        folders = ("datasets/actuacionesBomberos", "datasets/estaciones", "datasets/accidentalidad")
        for folder in folders:
            df = None
            for file in os.listdir(folder):
                filepath = folder + "/" + file
                df1 = pd.read_csv(filepath, sep=';', encoding='utf-8', low_memory=False)
                df = pd.concat([df, df1])
            self.data[str(folder)] = df

        for file_name in csv_files:
            file_path = os.path.join(self.folder_path, file_name)
            try:
                df = pd.read_csv(file_path, sep=';', encoding='latin-1', low_memory=False)
                self.data[str(file_name)] = df
                self.filename.append(file_name)
            except Exception as e:
                print(f"Error al leer {file_name}: {str(e)}")

        for value in self.data.keys():
            self.keys.append(value)

    def clean_data(self):
        """
        Cleans the loaded CSV data by renaming columns, removing whitespace, dropping null values and duplicates, and converting date columns to datetime format.

        Returns:
        --------
        None
        """
        columna_borrar = "Unnamed"
        for df in self.data:
            for j in self.data[df].columns:
                if columna_borrar in j:
                    while j in self.data[df].columns:
                        self.data[df] = self.data[df].drop(j, axis=1)
                        self.data[df] = self.data[df].dropna(how='all', axis=0)
                        
            self.data[df] = self.data[df].rename(columns = lambda x: x.strip().lower().replace(' ', '_'))
            self.data[df] = self.data[df].map(lambda x: x.strip() if isinstance(x, str) else x)
            self.data[df] = self.data[df].dropna(how='all', axis=0)
            self.data[df] = self.data[df].drop_duplicates()
            self.data[df] = self.data[df].loc[:, ~self.data[df].columns.duplicated()]
            self.data[df].columns = map(str.upper, self.data[df].columns)

            if 'FECHA' in self.data[df].columns:
                self.data[df]['FECHA'] = pd.to_datetime(self.data[df]['FECHA'], format='%d/%m/%Y')

#           num_cols = self.data[i].select_dtypes(include='number').columns
#           for col in num_cols:
#               self.data[i][col] = self.data[i][col].fillna(self.data[i][col].mean())

    def get_info(self, filename):
        print(self.data[filename].isnull().sum())
        print(self.data[filename].info())
        
    def get_nan_columns(self):
        j = 0
        for i in self.data:
            print(self.keys[j])
            self.get_info(i)
            j+=1
      
    def get_cleaned_data(self):
        """
        Returns the cleaned CSV data as a dictionary.

        Returns:
        --------
        dict
            A dictionary containing the cleaned CSV data, where the keys are the file names and the values are the corresponding dataframes.
        """
        return self.data

    def create_graph(df, colummn, name):
        frec = df[''+str(colummn)].value_counts()
        aux_df = pd.DataFrame(frec)
        aux_df.columns = ["Frecuencia absoluta"]
        aux_df["Frecuencia relativa"] = 100*aux_df["Frecuencia absoluta"] / len(df)
        frec_rel_cumsum = aux_df["Frecuencia relativa"].cumsum()
        aux_df["Frecuencia relativa acumulada"] = frec_rel_cumsum
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.set_title('Distribución de '+ str(name))
        ax.bar(aux_df.index, aux_df['Frecuencia absoluta'], color='blue')
        ax2 = ax.twinx()
        ax2.plot(aux_df.index, aux_df['Frecuencia relativa acumulada'], color='red', marker='o', ms = 5)
        ax2.yaxis.set_major_formatter(PercentFormatter())
        ax.tick_params(axis='y', color = 'blue')
        ax2.tick_params(axis='y', color = 'red')
        ax.set_xticklabels(aux_df.index, rotation=90)
        plt.show()

    def dataframe_summary(self, filename):

        numeric_mask = self.data[filename].select_dtypes(include='number').columns

        # Create a DataFrameSummary object
        summary = DataFrameSummary(self.data[filename][numeric_mask])
    
        # Display summary statistics
        summary_stats = summary.summary()
    
        # Plot correlation matrix for all numeric columns
        self.data[filename][numeric_mask].corr(method='pearson', numeric_only=True)

        # Boxplot of each variable
        #self.data[filename][numeric_mask].boxplot(figsize=(10, 8))

        return summary_stats

Una vez está definida la clase con sus métodos, procedemos a declarar las variables que nos permiten trabajar con ello

In [ ]:
folder_path = "datasets"
data_loader = CSVDataLoader(folder_path)

La siguiente función carga los datos de los csv a los dataframes

In [ ]:
data_loader.load_data()

Limpiamos los datos eliminando las columnas autogeneradas con NaNs, renombramos las columnas para que sean uniformes (minúsculas y con barra bajas), eliminamos las filas de NaNs, eliminamos las filas duplicadas, formateamos todas las variables fecha para que sean consistentes (dd/mm/aaaa), sustituimos los NaNs de las variables numéricas con la media correspondiente a su variable.

In [ ]:
data_loader.clean_data()
data = data_loader.get_cleaned_data()

Por último, para confirmar que los datos se han cargado bien, utilizamos el método get_nan_columns para ver cuántos datos en cada columna quedan nulos, así como usamos el método info() de pandas para ver un resumen de todas las columnas y comprobamos también que todo el formateo de las columnas se ha realizado sin problema.

In [ ]:
data_loader.get_nan_columns()

Efectivamente, se realiza sin problemas. El siguiente paso es el análisis de los datasets columna a columna para revisar qué método usar para rellenar los datos faltantes dependiendo de cada atributo.  

### Sustitución de valores faltantes
Los datasets actuaciones bomberos y estaciones no tienen datos faltantes, por lo que sólo nos queda trabajar con los otros 8 datasets, que podemos observar aquí:

In [ ]:
for i in data.keys():
    print(i)

Empecemos con accidentalidad: 

In [ ]:
data_loader.get_info('datasets/accidentalidad')
data_loader.data['datasets/accidentalidad']

Hay algunos datos faltantes que tiene sentido que lo sean, y podemos sustituir por un buzzword de algún tipo que nos haga saber que se trate de esto, como lo es que en un accidente en el que no se han producido lesiones, la lesividad sea nula y tampoco haya código de la misma, y como ambas cifras coinciden, es lógico pensar que se trata de las mismas situaciones. Podemos sustituir entonces todos los NaNs de Lesividad faltantes por 'Se desconoce' y el código por 77.

In [ ]:
print(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'].unique())

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==1)


In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==2)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==3)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==4)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==5)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==6)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==7)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==14)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==77)

In [ ]:
# Mapping dictionary
mapping_dict = {
    1: 'Atención en urgencias sin posterior ingreso',
    2: 'Ingreso inferior o igual a 24 horas',
    3: 'Ingreso superior a 24 horas',
    4: 'Fallecido 24 horas',
    5: 'Asistencia sanitaria ambulatoria con posterioridad',
    6: 'Asistencia sanitaria inmediata en centro de salud o mutua',
    7: 'Asistencia sanitaria sólo en el lugar del accidente',
    14: 'Sin asistencia sanitaria',
    77: 'Se desconoce',
}

# Fill missing values using the mapping dictionary
data_loader.data['datasets/accidentalidad']['LESIVIDAD'] = data_loader.data['datasets/accidentalidad']['LESIVIDAD'].fillna(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'].map(mapping_dict))
data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'] = data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'].fillna(data_loader.data['datasets/accidentalidad']['LESIVIDAD'].map({v: k for k, v in mapping_dict.items()}))

data_loader.data['datasets/accidentalidad']['LESIVIDAD'].fillna('Se desconoce', inplace=True)
data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'].fillna(77, inplace=True)


Por otro lado, positivo en droga tiene valor sólo si daba positivo, por lo que rellenar los valores faltantes con 0 es lo más lógico (siendo 0 negativo en droga). 

In [ ]:
data_loader.data['datasets/accidentalidad']['POSITIVA_DROGA'] = data_loader.data['datasets/accidentalidad']['POSITIVA_DROGA'].fillna(0)

Número, código de distrito, tipo de accidente, coordenadas (x e y), son atributos a los que sólo les falta un dato cada uno, por lo que no es representativo esta falta de datos y podemos rellenarlos con el valor más habitual. 

In [ ]:
numero = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
codDistrito = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
distrito = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
accidente = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
coorX = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
coorY = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]


data_loader.data['datasets/accidentalidad']['NUMERO'] = data_loader.data['datasets/accidentalidad']['NUMERO'].fillna(numero)
data_loader.data['datasets/accidentalidad']['COD_DISTRITO'] = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].fillna(codDistrito)
data_loader.data['datasets/accidentalidad']['DISTRITO'] = data_loader.data['datasets/accidentalidad']['DISTRITO'].fillna(distrito)
data_loader.data['datasets/accidentalidad']['TIPO_ACCIDENTE'] = data_loader.data['datasets/accidentalidad']['TIPO_ACCIDENTE'].fillna(accidente)
data_loader.data['datasets/accidentalidad']['COORDENADA_X_UTM'] = data_loader.data['datasets/accidentalidad']['COORDENADA_X_UTM'].fillna(coorX)
data_loader.data['datasets/accidentalidad']['COORDENADA_Y_UTM'] = data_loader.data['datasets/accidentalidad']['COORDENADA_Y_UTM'].fillna(coorY)


Por último, donde queda dilema es en positivo alcohol, tipo de vehículo y estado meteorológico. En esta situación, lo más apropiado es ver si, relacionando estos atributos con algún otro, es más probable que los atributos valgan uno u otro valor.

- Estado meteorológico. Hay 7 valores posibles: despejado, lluvia débil, lluvia intensa, granizando, nevando, nublado, se desconoce. Aquí, por lo tanto, hay 3 vías de actuación:
    - Rellenar con "se desconoce", i.e.: ser fieles a lo que se sabe, reducir la proporción de datos artificiales (hay un 11% de datos faltantes), solución sencilla.
    - Rellenar con el valor más frecuente: despejado (representa el 75% de los datos), i.e.: solución con datos artificiales más sencilla.
    - Rellenar con valores aleatorios según la proporción en la que aparecen los datos, i.e.: el 75% de los datos faltantes se rellenan arbitrariamente con "Despejado".

    Lo que mejor preserva los datos es, rellenar con "se desconoce", pues la variable existe previamente.

In [ ]:
data_loader.data['datasets/accidentalidad']['ESTADO_METEOROLÓGICO'] = data_loader.data['datasets/accidentalidad']['ESTADO_METEOROLÓGICO'].fillna('Se desconoce')

- Tipo de vehículo. Hay 34 valores posibles: Ambulancia SAMUR, autobús EMT, autobús, autobús articulado, autobús articulado EMT, autocaravana, bicicleta, bicicleta EPAC (pedaleo asistido), camión de bomberos, camión rígido, ciclo, ciclomotor, ciclomotor de dos ruedas L1e-B, cuadriciclo ligero, cuadriciclo no ligero, furgoneta, maquinaria de obras, microbús <= 17 plazas, moto de tres ruedas > 125cc, moto de tres ruedas hasta 125cc, motocicleta > 125cc, motocicleta hasta 125cc, otros vehículos con motor, otros vehículos sin motor, patinete no eléctrico, remolque, semirremolque, sin especificar, todo terreno, tractocamión, tren/metro, turismo (68%), VMU eléctrico, vehículo articulado. En esta variable hay 0.6% de valores faltantes, lo cual no es significativo, i.e.: la sustitución que elijamos tendrá menos repercusión en el estudio final. Aquí, por lo tanto, hay 2 vías de actuación:
    - Rellenar con "sin especificar", i.e.: solución sencilla y descriptiva pero que puede dar lugar a interpretaciones erróneas, pues puede haber sido otro tipo de vehículo que no se había registrado.
    - Rellenar con el valor más probable según otro atributo (por ejemplo, código de lesividad).
    
    La solución más apropiada es rellenar con el valor más probable según código de lesividad, por lo que vamos a ver primero cómo se relacionan ambos atributos y después rellenaremos los valores faltantes con el valor más probable.

In [ ]:
data_loader.data['datasets/accidentalidad']['TIPO_VEHICULO'] = data_loader.data['datasets/accidentalidad']['TIPO_VEHICULO'].fillna(data_loader.data['datasets/accidentalidad'].groupby('COD_LESIVIDAD')['TIPO_VEHICULO'].transform(lambda x:x.mode().iat[0]))

Para rellenar los datos de positivo en alcohol, usaremos lo más habitual según grupo de edad, sexo y lesividad de la persona

In [ ]:
data_loader.data['datasets/accidentalidad']["POSITIVA_ALCOHOL"] = data_loader.data['datasets/accidentalidad'].groupby(['RANGO_EDAD','SEXO','LESIVIDAD'])['POSITIVA_ALCOHOL'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty                                                                                                        else "Empty"))

Para determinar el tipo de persona, usaremos de criterio el rango de edad y la lesividad -si bien sólo faltan 3 datos, en esta situación podemos realizar un ajuste más específico-

In [ ]:
data_loader.data['datasets/accidentalidad']["TIPO_PERSONA"] = data_loader.data['datasets/accidentalidad'].groupby(['RANGO_EDAD','LESIVIDAD'])['TIPO_PERSONA'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty                                                                                                        else "Empty"))

In [ ]:
data_loader.get_info('datasets/accidentalidad')

Como podemos comprobar, no quedan datos faltantes. Pasamos a Actuaciones Bomberos:

In [ ]:
data_loader.get_info('datasets/actuacionesBomberos')

Este dataset no tiene datos faltantes, pasamos al siguiente. Estaciones:

In [ ]:
data_loader.get_info('datasets/estaciones')
data_loader.data['datasets/estaciones']

Este dataset tampoco tiene datos faltantes. Apreciamos que FSEN, aparte de indicar el sentido del tráfico, también indica si son las primeras 12h del día (1- y 2-), o las últimas (1=, 2=). Hacemos 12 nuevos atributos para suplir esto en un formato 24h que es más cómodo.

In [ ]:
# Convert 'date' column to datetime if it's not already
data_loader.data['datasets/estaciones']['FDIA'] = pd.to_datetime(data_loader.data['datasets/estaciones']['FDIA'], format='mixed', dayfirst=True)

In [ ]:
data_loader.get_info('datasets/estaciones')

In [ ]:
df = data_loader.data['datasets/estaciones']
# Split the FSEN values into direction and hour parts
df[['FSEN_DIRECTION', 'FSEN_AMPM']] = df['FSEN'].str.extract(r'(\d)([=-])')

# Drop unnecessary columns
df.drop(['FSEN'], axis=1, inplace=True)

index = ['FDIA', 'FEST', 'FSEN_DIRECTION', 'FSEN_AMPM', 'HOR1', 'HOR2', 'HOR3', 'HOR4', 'HOR5', 'HOR6', 'HOR7', 'HOR8', 'HOR9', 'HOR10', 'HOR11', 'HOR12', 'HOR13', 'HOR14', 'HOR15', 'HOR16', 'HOR17', 'HOR18', 'HOR19', 'HOR20', 'HOR21', 'HOR22', 'HOR23', 'HOR24']

df.reindex(columns=index)

In [ ]:
def group_fn(dff: pd.DataFrame):
    # dff contiene 2 rows de am y pm
    am = dff.loc[dff['FSEN_AMPM'] == "-"].iloc[0]
    pm = dff.loc[dff['FSEN_AMPM'] == "="].iloc[0]
    par = am[['HOR1', 'HOR2', 'HOR3', 'HOR4', 'HOR5', 'HOR6', 'HOR7', 'HOR8', 'HOR9', 'HOR10', 'HOR11', 'HOR12']].copy()
    
    par['HOR13'] = pm['HOR1']
    par['HOR14'] = pm['HOR2']
    par['HOR15'] = pm['HOR3']
    par['HOR16'] = pm['HOR4']
    par['HOR17'] = pm['HOR5']
    par['HOR18'] = pm['HOR6']
    par['HOR19'] = pm['HOR7']
    par['HOR20'] = pm['HOR8']
    par['HOR21'] = pm['HOR9']
    par['HOR22'] = pm['HOR10']
    par['HOR23'] = pm['HOR11']
    par['HOR24'] = pm['HOR12']

    return par

In [ ]:
data_loader.data['datasets/estaciones'] = df.groupby(['FEST', 'FDIA', 'FSEN_DIRECTION']).apply(group_fn)
data_loader.data['datasets/estaciones']

In [ ]:
data_loader.get_info('datasets/estaciones')

In [ ]:
data_loader.dataframe_summary('datasets/estaciones')

In [ ]:
data_loader.get_info('DireccionesEvolucionHistorica_20231004.csv')

Como podemos observar, faltan datos de las columnas VIA_SQC (secuencia de la denominación), con 371584 NaNs; VIA_PAR (partícula de la denominación), con 17997; CALIFICADOR (del número de la policía), con 322826; FECHA_DE_BAJA (de la dirección), con 211657; y TIPO_NDP (tipo de número: portal, garaje, fachada, etc.) con 16248.

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']

La secuencia de denominación (VIA_SQC) es solo NaNs, por lo que lo más lógico es eliminar la columna directamente

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv'].drop('VIA_SQC', axis=1)

Ahora, pasamos a la partícula de denominación (VIA_PAR), hay 7 valores distintos de entre la lista posible de 20 (DE, EL, A, DEL, LA, AL, DE LA, LAS, A LA, DE LAS, LO, A LAS, DE LO, LOS, A LO, DE LO, DE LOS, POR EL, A LOS, POR LA), de los cuales el primero es A LA y el último es DEL. Por lo tanto, parece que no se ha rellenado con Null o espacio vacío aquellas direcciones en las que no hay partícula:

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['VIA_PAR'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['VIA_PAR'].fillna(' ')

Fecha de baja, si no hay valor, es que está aún de alta, así que también sustituimos por un valor que represente que sigue dado de alta:

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['FECHA_DE_BAJA'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['FECHA_DE_BAJA'].fillna('--/--/----')

Para Tipo de número de policía, rellenaremos con el dato más habitual: PORTAL

In [ ]:
tipoNDP = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['TIPO_NDP'].mode().iat[0]
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['TIPO_NDP'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['TIPO_NDP'].fillna(tipoNDP)

Para Calificador, como no sabemos realmente cuál es el criterio y la mayoría de datos son faltantes (322826 faltan de 371584), lo más prudente es eliminar la columna directamente

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv'].drop('CALIFICADOR', axis=1)

In [ ]:
data_loader.get_info('DireccionesEvolucionHistorica_20231004.csv')

Efectivamente no quedan Missing Values. Pasamos a las direcciones vigentes:

In [ ]:
data_loader.get_info('DireccionesVigentes_20231004.csv')

Faltan valores en Vía Par (3381) y en Calificador (178838), los sustituimos con el mismo criterio que en el dataset anterior:

In [ ]:
data_loader.data['DireccionesVigentes_20231004.csv']['VIA_PAR'] = data_loader.data['DireccionesVigentes_20231004.csv']['VIA_PAR'].fillna(' ')

In [ ]:
data_loader.data['DireccionesVigentes_20231004.csv'] = data_loader.data['DireccionesVigentes_20231004.csv'].drop('CALIFICADOR', axis=1)

In [ ]:
data_loader.get_info('DireccionesVigentes_20231004.csv')

Pasamos al dataset estrella: radares.

In [ ]:
data_loader.get_info('RADARES_FIJOS_vDTT.csv')
data_loader.data['RADARES_FIJOS_vDTT.csv']

Carretera o vial, PK, sentido y tipo tienen muy poca frecuencia de NaNs, por lo que solo sustituimos por los valores más frecuentes

In [ ]:
carretera = data_loader.data['RADARES_FIJOS_vDTT.csv']['CARRETARA_O_VIAL'].mode().iat[0]
pk = data_loader.data['RADARES_FIJOS_vDTT.csv']['PK'].mode().iat[0]
sentido = data_loader.data['RADARES_FIJOS_vDTT.csv']['SENTIDO'].mode().iat[0]
tipo = data_loader.data['RADARES_FIJOS_vDTT.csv']['TIPO'].mode().iat[0]

data_loader.data['RADARES_FIJOS_vDTT.csv']['CARRETARA_O_VIAL'] = data_loader.data['RADARES_FIJOS_vDTT.csv']['CARRETARA_O_VIAL'].fillna(carretera)
data_loader.data['RADARES_FIJOS_vDTT.csv']['PK'] = data_loader.data['RADARES_FIJOS_vDTT.csv']['PK'].fillna(pk)
data_loader.data['RADARES_FIJOS_vDTT.csv']['SENTIDO'] = data_loader.data['RADARES_FIJOS_vDTT.csv']['SENTIDO'].fillna(sentido)
data_loader.data['RADARES_FIJOS_vDTT.csv']['TIPO'] = data_loader.data['RADARES_FIJOS_vDTT.csv']['TIPO'].fillna(tipo)

In [ ]:
data_loader.get_info('RADARES_FIJOS_vDTT.csv')

In [ ]:
data_loader.data['RADARES_FIJOS_vDTT.csv'].loc[11, 'X_(WGS84)'] = 441011.00
data_loader.data['RADARES_FIJOS_vDTT.csv'].loc[11, 'Y_(WGS84)'] = 4470920.00

data_loader.data['RADARES_FIJOS_vDTT.csv'].loc[14, 'X_(WGS84)'] = 439732.00
data_loader.data['RADARES_FIJOS_vDTT.csv'].loc[14, 'Y_(WGS84)'] = 4470920.00

A continuación identificamos los faltantes en la evolución histórica de viales:


In [ ]:
data_loader.get_info('VialesEvolucionHistorica_20231004.csv')
data_loader.data['VialesEvolucionHistorica_20231004.csv']

Faltan VIA_PAR y FECHA_DE_BAJA, que ya se han tratado en dataframes previos y manejaremos de la misma manera:

In [ ]:
data_loader.data['VialesEvolucionHistorica_20231004.csv']['VIA_PAR'] = data_loader.data['VialesEvolucionHistorica_20231004.csv']['VIA_PAR'].fillna(' ')
data_loader.data['VialesEvolucionHistorica_20231004.csv']['FECHA_DE_BAJA'] = data_loader.data['VialesEvolucionHistorica_20231004.csv']['FECHA_DE_BAJA'].fillna('--/--/----')

In [ ]:
data_loader.get_info('VialesEvolucionHistorica_20231004.csv')

Pasamos al dataframe de viales vigentes distritos barrios:

In [ ]:
data_loader.get_info('VialesVigentesDistritosBarrios_20231004.csv')
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']

VIA_PAR hacemos la misma solución que previamente:

In [ ]:
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['VIA_PAR'] = data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['VIA_PAR'].fillna(' ')

PAR_MIN, PAR_MAX, IMPAR_MAX e IMPAR_MIN tienen un valor para los casos en los que no hay calificador: '-', y es lo que usaremos para sustituir los datos faltantes.

In [ ]:
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['IMPAR_MAX'] = data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['IMPAR_MAX'].fillna('-')
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['IMPAR_MIN'] = data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['IMPAR_MIN'].fillna('-')
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['PAR_MAX'] = data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['PAR_MAX'].fillna('-')
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['PAR_MIN'] = data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['PAR_MIN'].fillna('-')

Pasamos a viales vigentes distritos:

In [ ]:
data_loader.get_info('VialesVigentesDistritos_20231004.csv')
data_loader.data['VialesVigentesDistritos_20231004.csv']

In [ ]:
data_loader.data['VialesVigentesDistritos_20231004.csv']['VIA_PAR'] = data_loader.data['VialesVigentesDistritos_20231004.csv']['VIA_PAR'].fillna(' ')
data_loader.data['VialesVigentesDistritos_20231004.csv']['IMPAR_MAX'] = data_loader.data['VialesVigentesDistritos_20231004.csv']['IMPAR_MAX'].fillna('-')
data_loader.data['VialesVigentesDistritos_20231004.csv']['IMPAR_MIN'] = data_loader.data['VialesVigentesDistritos_20231004.csv']['IMPAR_MIN'].fillna('-')
data_loader.data['VialesVigentesDistritos_20231004.csv']['PAR_MAX'] = data_loader.data['VialesVigentesDistritos_20231004.csv']['PAR_MAX'].fillna('-')
data_loader.data['VialesVigentesDistritos_20231004.csv']['PAR_MIN'] = data_loader.data['VialesVigentesDistritos_20231004.csv']['PAR_MIN'].fillna('-')

Pasamos al último dataset: viales vigentes a 4/10/2023

In [ ]:
data_loader.get_info('VialesVigentes_20231004.csv')
data_loader.data['VialesVigentes_20231004.csv']

Particula termina y partícula comienza está estrechamente relacionada con vía par, por lo que la sustitución de valores es igual

In [ ]:
data_loader.data['VialesVigentes_20231004.csv']['VIA_PAR'] = data_loader.data['VialesVigentes_20231004.csv']['VIA_PAR'].fillna(' ')
data_loader.data['VialesVigentes_20231004.csv']['PARTICULA_TERMINA'] = data_loader.data['VialesVigentes_20231004.csv']['PARTICULA_TERMINA'].fillna(' ')
data_loader.data['VialesVigentes_20231004.csv']['PARTICULA_COMIENZA'] = data_loader.data['VialesVigentes_20231004.csv']['PARTICULA_COMIENZA'].fillna(' ')

### Análisis Descriptivo de los Datos
A continuación, veremos, dataframe a dataframe, el análisis descriptivo


In [ ]:
for i in data.keys():
    print(i)

Empezando con el dataframe de actuaciones bomberos:

In [ ]:
data_loader.dataframe_summary('datasets/actuacionesBomberos')

In [ ]:
pd.DataFrame.boxplot(data_loader.data['datasets/actuacionesBomberos'], column=['FUEGOS', 'DAÑOS_EN_CONSTRUCCION', 'SALVAMENTOS_Y_RESCATES','DAÑOS_POR_AGUA','INCIDENTES_DIVERSOS'], figsize=[15, 8])

In [ ]:
horas = ['HOR1', 'HOR2','HOR3', 'HOR4', 'HOR5', 'HOR6', 'HOR7', 'HOR8', 'HOR9', 'HOR10' , 'HOR11', 'HOR12', 'HOR13', 'HOR14', 'HOR15', 'HOR16', 'HOR17', 'HOR18', 'HOR19', 'HOR20', 'HOR21', 'HOR22', 'HOR23', 'HOR24']
#data_loader.data['datasets/estaciones'].plot.bar(x=horas, y=)

In [ ]:
data_loader.get_info('datasets/accidentalidad')

In [ ]:
data_loader.dataframe_summary('datasets/estaciones')

In [ ]:
data_loader.dataframe_summary('DireccionesEvolucionHistorica_20231004.csv')

In [ ]:
data_loader.dataframe_summary('DireccionesVigentes_20231004.csv')

In [ ]:
data_loader.dataframe_summary('iluminacion.csv')

In [ ]:
data_loader.dataframe_summary('padron22.csv')

In [ ]:
data_loader.dataframe_summary('RADARES_FIJOS_vDTT.csv')

In [ ]:
data_loader.dataframe_summary('VialesEvolucionHistorica_20231004.csv')

In [ ]:
data_loader.dataframe_summary('VialesVigentesDistritosBarrios_20231004.csv')

In [ ]:
data_loader.dataframe_summary('VialesVigentesDistritos_20231004.csv')

In [ ]:
data_loader.dataframe_summary('VialesVigentes_20231004.csv')


Tabla de frecuencia relativa de accidentes por distritos. En esta tabla se obtiene la frecuencia de accidentes en todos los distritos de madrid, esto hace que se pueda obtener también el tráfico de coches en el distrito. Aunque no sea una relación proporcional, a mayor número de accidentes supone un tráfico mayor en la zona.

In [ ]:
data_loader.data['datasets/accidentalidad']['DISTRITO'] = data_loader.data['datasets/accidentalidad']['DISTRITO'].astype(str)
CSVDataLoader.create_graph(data_loader.data['datasets/accidentalidad'], 'DISTRITO', 'accidentes por distrito')

Tabla comparativa que muestra los accidentes donde la persona que lo ha padecido ha dado positivo en alcohol.

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']
positivos = accidentes[accidentes['POSITIVA_ALCOHOL'] == 'S']
negativos = accidentes[accidentes['POSITIVA_ALCOHOL'] == 'N']
labels = ['Positivo en Alcohol', 'Negativo en Alcohol', 'No se realizó la prueba']
values = [len(positivos), len(negativos), len(accidentes) - len(positivos) - len(negativos)]
plt.bar(labels, values)
plt.xlabel('Resultado de Alcohol')
plt.ylabel('Número de Accidentes')
plt.title('Comparación de Accidentes con y sin Positivo en Alcohol')
plt.show()

Tabla comparativo donde la persona que ha padecido el accidente ha dado positivo en drogas.

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']
positivos = accidentes[accidentes['POSITIVA_DROGA'] == 1]
negativos = accidentes[accidentes['POSITIVA_DROGA'] == 0]
labels = ['Positivo en Drogas', 'Negativo en Drogas']
values = [len(positivos), len(negativos)]
plt.bar(labels, values)
plt.xlabel('Resultado de Drogas')
plt.ylabel('Número de Accidentes')
plt.title('Comparación de Accidentes con y sin Positivo en Drogas')
plt.show()

Tabla comparativa que relaciona los accidente en los que la persona ha dado positivo en alcohol y drogas

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']
positivos_alcohol = accidentes[accidentes['POSITIVA_ALCOHOL'] == 'S']
positivos_drogas = accidentes[accidentes['POSITIVA_DROGA'] == 1]
positivos_alcohol_drogas = positivos_alcohol[positivos_alcohol['POSITIVA_DROGA'] == 1]
positivos_alcohol_drogas = len(positivos_alcohol_drogas)
negativos = len(accidentes) - positivos_alcohol_drogas
labels = ['Positivo en Alcohol y Drogas', 'Negativo en Alcohol y Drogas']
values = [positivos_alcohol_drogas, negativos]
plt.bar(labels, values)
plt.xlabel('Resultado de Alcohol')
plt.ylabel('Número de Accidentes')
plt.title('Comparación de Accidentes con y sin Positivo en Alcohol')
plt.show()

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']
labels = accidentes['TIPO_ACCIDENTE'].unique()
values = accidentes['TIPO_ACCIDENTE'].value_counts()
percentages = (values / len(accidentes)) * 100
plt.pie(values, shadow=True, startangle=90)
plt.title('Tipos de Accidentes')
legend_labels = [f'{label} ({percentage:.2f}%)' for label, percentage in zip(labels, percentages)]
plt.legend(labels=legend_labels, loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

In [ ]:
# Create a cross-tabulation of TIPO_PERSONA and TIPO_ACCIDENTE
cross_tab = pd.crosstab(accidentes['TIPO_PERSONA'], accidentes['TIPO_ACCIDENTE'])

# Plotting the heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(cross_tab, annot=True, cmap="YlGnBu", fmt='g')
plt.title('Relación tipo de accidente y tipo persona')
plt.xlabel('Accidente')
plt.ylabel('Persona')

plt.show()

In [ ]:
# Create a faceted plot
g = sns.FacetGrid(accidentes, col='LESIVIDAD', height=4, col_wrap=3)
g.map_dataframe(sns.countplot, x='TIPO_PERSONA', hue='TIPO_ACCIDENTE', palette='Set1')
g.set_axis_labels('TIPO_PERSONA', 'Count')
g.set_titles(col_template='{col_name}')

# Add a legend
g.add_legend(title='TIPO_ACCIDENTE')

plt.show()

In [ ]:
# Filter the DataFrame for cases where TIPO_PERSONA is 'Driver'
df_driver = accidentes[accidentes['TIPO_PERSONA'] == 'Conductor']

# Create a grouped bar plot
plt.figure(figsize=(10, 6))
plot = sns.countplot(data=df_driver, x='LESIVIDAD', hue='TIPO_ACCIDENTE', palette='Set1')

# Wrap the x-axis labels
wrapped_labels = [textwrap.fill(label.get_text(), 10) for label in plot.get_xticklabels()]
plot.set_xticklabels(wrapped_labels)

plt.title('Relación entre tipo de lesividad y tipo de accidente para conductor')
plt.xlabel('LESIVIDAD')
plt.ylabel('Count')
plt.legend(title='Accidente')

plt.show()

In [ ]:
df_dead = accidentes[accidentes['COD_LESIVIDAD'] == 4]

# Create a grouped bar plot
plt.figure(figsize=(10, 6))
plot = sns.countplot(data=df_dead, x='TIPO_PERSONA', hue='TIPO_ACCIDENTE', palette='Set1')

# Wrap the x-axis labels
wrapped_labels = [textwrap.fill(label.get_text(), 10) for label in plot.get_xticklabels()]
plot.set_xticklabels(wrapped_labels)

plt.title('Relación entre tipo de persona y tipo de accidente que resultó en fallecimiento')
plt.xlabel('LESIVIDAD')
plt.ylabel('Count')
plt.legend(title='Accidente')

plt.show()

In [ ]:
df_peaton = accidentes[(accidentes['TIPO_PERSONA'] == 'Peatón') & (accidentes['COD_LESIVIDAD'] == 4)]

# Plot the count of ESTADO_METEOROLOGICO
plt.figure(figsize=(8, 6))
sns.countplot(data=df_peaton, x='ESTADO_METEOROLÓGICO', palette='dark')
plt.title('Clima con víctimas mortales peatones')
plt.show()

In [ ]:
estaciones = data_loader.data['datasets/estaciones']


In [ ]:
direccionesvigentes = data_loader.data['DireccionesVigentes_20231004.csv']

In [ ]:
data_loader.get_info('padron22.csv')

In [ ]:
data_loader.data['padron22.csv'] = data_loader.data['padron22.csv'].rename(columns={'TIPO_VEHÃCULO': 'TIPO_VEHICULO'})
data_loader.data['padron22.csv'] = data_loader.data['padron22.csv'].rename(columns={'COD_TIPO_VEHÃCULO': 'COD_TIPO_VEHICULO'})

padron = data_loader.data['padron22.csv']
print(data_loader.data['padron22.csv']['COD_TIPO_VEHICULO'].unique())



In [ ]:
census_proportions = padron['TIPO_VEHICULO'].value_counts(normalize=True)

accidentalidad_proportions = accidentes['TIPO_VEHICULO'].value_counts(normalize=True)

value_mapping_dict = {
    'Autobús': 'AUTOBUS',
    'Autobus EMT': 'AUTOBUS',
    'Autobús articulado': 'AUTOBUS',
    'Autobús articulado EMT': 'AUTOBUS',
    'Camión de bomberos': 'CAMION',
    'Camión rígido': 'CAMION',
    'Ciclomotor': 'CICLOMOTOR',
    'Ciclomotor de tres ruedas': 'CICLOMOTOR',
    'Ciclomotor de dos ruedas L1e-B': 'CICLOMOTOR',
    'Moto de tres ruedas > 125cc': 'MOTOCICLETA',
    'Moto de tres ruedas hasta 125cc': 'MOTOCICLETA',
    'Motocicleta > 125cc': 'MOTOCICLETA',
    'Motocicleta hasta 125cc': 'MOTOCICLETA',
    'Remolque': 'REMOLQUE',
    'Semirremolque': 'SEMIRREMOLQUE',
    'Tractocamión': 'TRACTOR',
    'Turismo': 'TURISMO',
}

columnasAcc = ['TIPO_VEHICULO', 'COD_LESIVIDAD', 'TIPO_ACCIDENTE']
accidenteSubset = accidentes[columnasAcc]
padronSubset = padron['TIPO_VEHICULO']


accidenteSubset.loc[:, 'TIPO_VEHICULO'] = accidenteSubset['TIPO_VEHICULO'].replace(value_mapping_dict)

# Merge the datasets on the common column(s)
merged_data = pd.merge(accidenteSubset, padronSubset, on='TIPO_VEHICULO', how='inner')

# Create a DataFrame with proportions for the merged data
proportions_df = pd.DataFrame({
    'Accidentalidad': merged_data['TIPO_VEHICULO'].value_counts(normalize=True),
    'Census': merged_data['Proportion_Column'].value_counts(normalize=True)
})

proportions_df = pd.DataFrame({
    'Census': census_proportions,
    'Accidentalidad': accidentalidad_proportions
})

# Step 4: Compare Distributions using Heatmap
plt.figure(figsize=(12, 6))

sns.heatmap(proportions_df.T, annot=False, cmap="", cbar=True)
plt.title('Proportions of Vehicle Types in Census and Accidentalidad Datasets')


plt.show()

Este gráfico muestra los distintos tipos de accidentes registrados. Se puede ver que la colisión frontal-lateral y el alcance son las mauores causas de accidentes.

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']
labels = accidentes['LESIVIDAD'].unique()
values = accidentes['LESIVIDAD'].value_counts()
percentages = (values / len(accidentes)) * 100
plt.pie(values, shadow=True, startangle=90)
plt.title('Lesividad')
legend_labels = [f'{label} ({percentage:.2f}%)' for label, percentage in zip(labels, percentages)]
plt.legend(labels=legend_labels, loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

Este gráfico muestra el tipo de lesividad del accidente. Se aprecia que 3 de cada 4 personas no necesitan asistencia sanitaria.